In [1]:
''' Test accuracy-speed trade-off of cpp simulators with different robots, controllers and reference motions
'''
import time
import consim 
import numpy as np
from numpy.linalg import norm as norm

from example_robot_data.robots_loader import loadSolo, loadRomeo, getModelPath

import matplotlib.pyplot as plt 
import consim_py.utils.plot_utils as plut
import pinocchio as pin 
from pinocchio.robot_wrapper import RobotWrapper
import pickle

from consim_py.ral2020.simu_cpp_common import Empty, dt_ref, play_motion, load_solo_ref_traj, \
    plot_multi_x_vs_y_log_scale, compute_integration_errors, run_simulation, plot_multi_x_vs_y_rate_of_change

# CONTROLLERS
from consim_py.ral2020.linear_feedback_controller import LinearFeedbackController
from consim_py.tsid_quadruped import TsidQuadruped
from consim_py.ral2020.tsid_biped import TsidBiped

# 
from PIL import Image, ImageDraw, ImageFont

In [2]:
def ndprint(a, format_string ='{0:.2f}'):
    print([format_string.format(v,i) for i,v in enumerate(a)])
    
comp_times_exp    = ['exponential_simulator::step',
                     'exponential_simulator::substep',
                     'exponential_simulator::computeExpLDS',
                     'exponential_simulator::computeIntegralsXt',
                     'exponential_simulator::kinematics',
                     'exponential_simulator::forwardDynamics',
                     'exponential_simulator::resizeVectorsAndMatrices']
comp_times_euler = ['euler_simulator::step',
                    'euler_simulator::substep']
comp_times_implicit_euler = ['imp_euler_simulator::step',
                             'imp_euler_simulator::substep',
                             'imp_euler_simulator::computeDynamicsJacobian',
                             'imp_euler_simulator::Minv_JT_K_J',
                             'imp_euler_simulator::computeNonlinearEquations',
                             'imp_euler_simulator::computeNewtonSystem',
                             'imp_euler_simulator::solveNewtonSystem',
                             'imp_euler_simulator::lineSearch',
                             'imp_euler_simulator::computeABADerivatives',
                             'imp_euler_simulator::copyContacts']
comp_times_rk4   = ['rk4_simulator::step',
                    'rk4_simulator::substep']
                    
comp_times_exp_dict = {}
comp_times_euler_dict = {}
comp_times_implicit_euler_dict = {}
comp_times_rk4_dict = {}
for s in comp_times_exp:
    comp_times_exp_dict[s] = s.split('::')[-1]
for s in comp_times_euler:
    comp_times_euler_dict[s] = s.split('::')[-1]
for s in comp_times_implicit_euler:
    comp_times_implicit_euler_dict[s] = s.split('::')[-1]
for s in comp_times_rk4:
    comp_times_rk4_dict[s] = s.split('::')[-1]
    
plut.SAVE_FIGURES = 1
PLOT_FORCES = 0
PLOT_CONTACT_POINTS = 0
PLOT_VELOCITY_NORM = 0
PLOT_SLIPPING = 0
PLOT_BASE_POS = 0
PLOT_INTEGRATION_ERRORS = 1
PLOT_INTEGRATION_ERROR_TRAJECTORIES = 0
PLOT_MATRIX_MULTIPLICATIONS = 0
PLOT_MATRIX_NORMS = 0

LOAD_GROUND_TRUTH_FROM_FILE = 0
SAVE_GROUND_TRUTH_TO_FILE = 0
RESET_STATE_ON_GROUND_TRUTH = 1  # reset the state of the system on the ground truth

#TEST_NAME = 'solo-squat'
# TEST_NAME = 'solo-trot'
TEST_NAME = 'solo-jump'
#TEST_NAME = 'romeo-walk'
#TEST_NAME = 'talos-walk'

LINE_WIDTH = 100
print("".center(LINE_WIDTH, '#'))
print(" Test Consim C++ ".center(LINE_WIDTH, '#'))
print(TEST_NAME.center(LINE_WIDTH, '#'))
print("".center(LINE_WIDTH, '#'))

plut.FIGURE_PATH = './'+TEST_NAME+'/'
N = 300
dt = 0.010      # controller and simulator time step

# DEBUG
#N = 1
# END DEBUG

if(TEST_NAME=='solo-squat'):
    robot_name = 'solo'
    motionName = 'squat'
    ctrl_type = 'tsid-quadruped'
    com_offset = np.array([0.0, -0.0, 0.0])
    com_amp    = np.array([0.0, 0.0, 0.05])
    com_freq   = np.array([0.0, .0, 2.0])
if(TEST_NAME=='solo-trot'):
    robot_name = 'solo'
    motionName = 'trot'
    ctrl_type = 'linear'
    dt = 0.002      # controller and simulator time step
    assert(np.floor(dt_ref/dt)==dt_ref/dt)
if(TEST_NAME=='solo-jump'):
    robot_name = 'solo'
    motionName = 'jump'
    ctrl_type = 'linear'
    assert(np.floor(dt_ref/dt)==dt_ref/dt)
elif(TEST_NAME=='romeo-walk'):
    robot_name = 'romeo'
    motionName = 'walk'
    ctrl_type = 'tsid-biped'
    dt = 0.04
elif(TEST_NAME=='talos-walk'):
    robot_name = 'talos'
    motionName = 'walk'
    ctrl_type = 'tsid-biped'
    dt = 0.03

# ground truth computed with time step 1/64 ms
ground_truth_dt = 1e-3/64
i_ground_truth = int(np.log2(dt / ground_truth_dt))

i_min = 0
i_max = i_ground_truth - 2

GROUND_TRUTH_EXP_SIMU_PARAMS = {
    'name': 'ground-truth %d'%(2**i_ground_truth),
    'method_name': 'ground-truth-exp',
    'simulator': 'exponential',
    'ndt': 2**i_ground_truth,
}

SIMU_PARAMS = []

# EXPONENTIAL INTEGRATOR WITH STANDARD SETTINGS
for i in range(i_min, i_max):
    for m in [0]:#, 2, 4, -1]:
#    for m in [-1]:
        SIMU_PARAMS += [{
            'name': 'expo %4d mmm%2d'%(2**i,m),
            'method_name': 'Expo mmm%2d'%(m),
            'simulator': 'exponential',
            'ndt': 2**i,
            'forward_dyn_method': 3,
            'max_mat_mult': m
        }]
        
#i_min = 1
#i_max -= 3
#i_ground_truth = i_max+2
#i_ground_truth = 0
#dt = 0.002
#N = 10
#i_ground_truth = int(np.log2(dt / ground_truth_dt))

GROUND_TRUTH_IMPLICIT_EULER_SIMU_PARAMS = {
    'name': 'ground-truth %d'%(2**i_ground_truth),
    'method_name': 'ground-truth-implicit-euler',
    'simulator': 'implicit-euler',
    'ndt': 2**i_ground_truth,
    'use_finite_differences_dynamics': False,
    'use_finite_differences_nle': False,
    'use_current_state_as_initial_guess': False,
    'convergence_threshold': 1e-6
}
print("dt=", dt, "ndt", 2**i_ground_truth)
#i_max = 1

# IMPLICIT EULER INTEGRATOR
for i in range(i_min, i_max):
    SIMU_PARAMS += [{
        'name': 'imp-eul %4d'%(2**i),
        'method_name': 'Eul-imp',
        'simulator': 'implicit-euler',
        'ndt': 2**i,
        'use_finite_differences_dynamics': False,
        'use_finite_differences_nle': False,
        'use_current_state_as_initial_guess': False,
        'convergence_threshold': 1e-6
    }]
# IMPLICIT EULER INTEGRATOR with finite differences
#for i in range(i_min, i_max):
#    SIMU_PARAMS += [{
#        'name': 'imp-eul-fd %4d'%(2**i),
#        'method_name': 'imp-eul-fd',
#        'simulator': 'implicit-euler',
#        'ndt': 2**i,
#        'use_finite_differences_dynamics': False,
#        'use_finite_differences_nle': True,
#        'use_current_state_as_initial_guess': False,
#        'convergence_threshold': 1e-5
#    }]
    
GROUND_TRUTH_RK4_SIMU_PARAMS = {
    'name': 'ground-truth %d'%(2**i_ground_truth),
    'method_name': 'ground-truth-rk4',
    'simulator': 'rk4',
    'ndt': 2**i_ground_truth,
}

# RK4 INTEGRATOR
for i in range(i_min, i_max):
    SIMU_PARAMS += [{
        'name': 'rk4 %4d'%(2**i),
        'method_name': 'RK4',
        'simulator': 'rk4',
        'ndt': 2**i,
    }]
    
    

# i_min += 0
# i_max += 3
i_ground_truth = i_max+2
GROUND_TRUTH_EULER_SIMU_PARAMS = {
    'name': 'ground-truth %d'%(2**i_ground_truth),
    'method_name': 'ground-truth-euler',
    'simulator': 'euler',
    'ndt': 2**i_ground_truth,
    'integration_type': 0
}

# EULER INTEGRATOR WITH EXPLICIT INTEGRATION
for i in range(i_min, i_max):
    SIMU_PARAMS += [{
        'name': 'eul-ex %4d'%(2**i),
        'method_name': 'Eul-exp',
        'simulator': 'euler',
        'ndt': 2**i,
        'forward_dyn_method': 3,
        'integration_type': 0
    }]
    
# EULER INTEGRATOR WITH SEMI-IMPLICIT INTEGRATION
#for i in range(i_min, i_max):
#    SIMU_PARAMS += [{
#        'name': 'eul-semi%4d'%(2**i),
#        'method_name': 'eul-semi',
#        'simulator': 'euler',
#        'ndt': 2**i,
#        'forward_dyn_method': 3,
#        'integration_type': 1
#    }]
    
# EULER INTEGRATOR WITH CLASSIC EXPLICIT INTEGRATION
#for i in range(i_min, i_max):
#    SIMU_PARAMS += [{
#        'name': 'Eul-clexp%4d'%(2**i),
#        'method_name': 'Eul-clexp',
#        'simulator': 'euler',
#        'ndt': 2**i,
#        'forward_dyn_method': 3,
#        'integration_type': 2
#    }]

#for i in range(i_min, i_max):
#    SIMU_PARAMS += [{
#        'name': 'euler ABA%4d'%(2**i),
#        'method_name': 'euler ABA',
#        'simulator': 'euler',
#        'ndt': 2**i,
#        'forward_dyn_method': 2
#    }]
#
#for i in range(i_min, i_max):
#    SIMU_PARAMS += [{
#        'name': 'euler Chol%4d'%(2**i),
#        'method_name': 'euler Chol',
#        'simulator': 'euler',
#        'ndt': 2**i,
#        'forward_dyn_method': 3
#    }]


    
#unilateral_contacts = 1

####################################################################################################
######################################### Test Consim C++ ##########################################
#############################################solo-jump##############################################
####################################################################################################
dt= 0.01 ndt 512


In [3]:
if(robot_name=='solo'):
    import consim_py.ral2020.conf_solo_cpp as conf
    robot = loadSolo(False)
elif(robot_name=='romeo' or robot_name=='talos'):
    if(robot_name=='romeo'):
        import consim_py.ral2020.conf_romeo_cpp as conf
    elif(robot_name=='talos'):
        import consim_py.ral2020.conf_talos_cpp as conf
    robot = RobotWrapper.BuildFromURDF(conf.urdf, [conf.modelPath], pin.JointModelFreeFlyer())
    
    contact_point = np.ones((3,4)) * conf.lz
    contact_point[0, :] = [-conf.lxn, -conf.lxn, conf.lxp, conf.lxp]
    contact_point[1, :] = [-conf.lyn, conf.lyp, -conf.lyn, conf.lyp]
    contact_frames = []
    for cf in conf.contact_frames:
        parentFrameId = robot.model.getFrameId(cf)
        parentJointId = robot.model.frames[parentFrameId].parent
        for i in range(4):
            frame_name = cf+"_"+str(i)
            placement = pin.XYZQUATToSE3(list(contact_point[:,i])+[0, 0, 0, 1.])
            placement = robot.model.frames[parentFrameId].placement * placement
            fr = pin.Frame(frame_name, parentJointId, parentFrameId, placement, pin.FrameType.OP_FRAME)
            robot.model.addFrame(fr)
            contact_frames += [frame_name]
    conf.contact_frames = contact_frames
    robot.data = robot.model.createData()

PRINT_N = int(conf.PRINT_T/dt)
ground_truth_file_name = robot_name+"_"+motionName+str(dt)+"_cpp.p"

nq, nv = robot.nq, robot.nv

if conf.use_viewer:
    robot.initViewer(loadModel=True)
    robot.viewer.gui.createSceneWithFloor('world')
    robot.viewer.gui.setLightingMode('world/floor', 'OFF')

# create feedback controller
if(ctrl_type=='linear'):
    refX, refU, feedBack = load_solo_ref_traj(robot, dt, motionName)
    controller = LinearFeedbackController(robot, dt, refX, refU, feedBack)
    q0, v0 = controller.q0, controller.v0
    N = controller.refU.shape[0]
elif(ctrl_type=='tsid-quadruped'):
    controller = TsidQuadruped(conf, dt, robot, com_offset, com_freq, com_amp, conf.q0, viewer=False)
    q0, v0 = conf.q0, conf.v0
elif(ctrl_type=='tsid-biped'):
    controller = TsidBiped(conf, dt, conf.urdf, conf.modelPath, conf.srdf)
    q0, v0 = controller.q0, controller.v0
    N = controller.N+int((conf.T_pre+conf.T_post)/dt)


if(LOAD_GROUND_TRUTH_FROM_FILE):    
    print("\nLoad ground truth from file")
    data = pickle.load( open( ground_truth_file_name, "rb" ) )
    
#    i0, i1 = 1314, 1315
#    refX = refX[i0:i1+1,:]
#    refU = refU[i0:i1,:]
#    feedBack = feedBack[i0:i1,:,:]
##    q0, v0 = refX[0,:nq], refX[0,nq:]
#    N = refU.shape[0]
#    for key in ['ground-truth-exp', 'ground-truth-euler']:
#        data[key].q  = data[key].q[:,i0:i1+1]
#        data[key].v  = data[key].v[:,i0:i1+1]
#        data[key].p0 = data[key].p0[:,:,i0:i1+1]
#        data[key].slipping = data[key].slipping[:,i0:i1+1]
#    q0, v0 = data['ground-truth-exp'].q[:,0], data['ground-truth-exp'].v[:,0]
else:
    data = {}
    print("\nStart simulation ground truth Implicit Euler")
    data['ground-truth-implicit-euler'] = run_simulation(conf, dt, N, robot, controller, q0, v0, GROUND_TRUTH_IMPLICIT_EULER_SIMU_PARAMS)
    
    print("\nStart simulation ground truth RK4")
    data['ground-truth-rk4'] = run_simulation(conf, dt, N, robot, controller, q0, v0, GROUND_TRUTH_RK4_SIMU_PARAMS)
    
    print("\nStart simulation ground truth Exponential")
    data['ground-truth-exp'] = run_simulation(conf, dt, N, robot, controller, q0, v0, GROUND_TRUTH_EXP_SIMU_PARAMS)
    
    print("\nStart simulation ground truth Euler")
    data['ground-truth-euler'] = run_simulation(conf, dt, N, robot, controller, q0, v0, GROUND_TRUTH_EULER_SIMU_PARAMS)
    
    if(SAVE_GROUND_TRUTH_TO_FILE):
        pickle.dump( data, open( ground_truth_file_name, "wb" ) )



Start simulation ground truth Implicit Euler

Start simulation ground truth RK4

Start simulation ground truth Exponential

Start simulation ground truth Euler


In [4]:
for simu_params in SIMU_PARAMS:
    name = simu_params['name']
    print("\nStart simulation", name)
    if(simu_params['simulator']=='exponential'):
        data[name] = run_simulation(conf, dt, N, robot, controller, q0, v0, simu_params, 
                                    data['ground-truth-exp'], comp_times_exp_dict)
    elif(simu_params['simulator']=='euler'):
        data[name] = run_simulation(conf, dt, N, robot, controller, q0, v0, simu_params, 
                            data['ground-truth-euler'], comp_times_euler_dict)
    elif(simu_params['simulator']=='implicit-euler'):
        data[name] = run_simulation(conf, dt, N, robot, controller, q0, v0, simu_params, 
                            data['ground-truth-implicit-euler'], comp_times_implicit_euler_dict)
        print("Avg iter: %.1f"%(np.mean(data[name].avg_iter_num)))
#        print("Comp time:", data[name].computation_times['step'].avg)
    elif(simu_params['simulator']=='rk4'):
        data[name] = run_simulation(conf, dt, N, robot, controller, q0, v0, simu_params, 
                            data['ground-truth-rk4'], comp_times_rk4_dict)

# COMPUTE INTEGRATION ERRORS:
res = compute_integration_errors(data, robot, dt)

# PLOT STUFF
line_styles = 100*['-o', '--o', '-.o', ':o']
tt = np.arange(0.0, (N+1)*dt, dt)[:N+1]
descr_str = "k_%.1f_b_%.1f"%(np.log10(conf.K[0]), np.log10(conf.B[0]))


Start simulation expo    1 mmm 0

Start simulation expo    2 mmm 0

Start simulation expo    4 mmm 0
Exception while running simulation Time 2.470 Velocities are too large: 19834776.4. Stop simulation.

Start simulation expo    8 mmm 0
Exception while running simulation Time 2.270 Velocities are too large: 1524620.2. Stop simulation.

Start simulation expo   16 mmm 0

Start simulation expo   32 mmm 0

Start simulation expo   64 mmm 0

Start simulation imp-eul    1
Exception while running simulation Time 2.570 Velocities are too large: 114406820.8. Stop simulation.
Avg iter: 10.1

Start simulation imp-eul    2
Exception while running simulation Time 2.870 Velocities are too large: 331583884.2. Stop simulation.
Avg iter: 7.7

Start simulation imp-eul    4
Avg iter: 6.2

Start simulation imp-eul    8
Avg iter: 4.6

Start simulation imp-eul   16
Avg iter: 3.6

Start simulation imp-eul   32
Avg iter: 3.3

Start simulation imp-eul   64
Avg iter: 2.7

Start simulation rk4    1
Exception whil

In [5]:
import consim_py.utils.visualize as visualize 
from example_robot_data.robots_loader import getModelPath

if(TEST_NAME[:4]=='solo'):
    urdf_path = conf.urdf 
    model_path = conf.modelPath +'/../../../..'
    
if(TEST_NAME=='solo-jump'):
    CameraTransform =[-1.5264389514923096,
     -1.6716818809509277,
     0.2570425570011139,
     0.6916553378105164,
     -0.23205697536468506,
     -0.21380648016929626,
     0.6496531367301941]
if(TEST_NAME=='solo-trot'):
    CameraTransform =[1.688956618309021,
     -2.023496389389038,
     0.4910387396812439,
     0.6164828538894653,
     0.17589177191257477,
     0.191686749458313,
     0.7431467771530151]

if(TEST_NAME=='romeo-walk'):
#     CameraTransform = [0.3461300730705261,
#              -2.874166488647461,
#              0.4731728136539459,
#              0.733085036277771,
#              0.006836195942014456,
#              0.005807977169752121,
#              0.6800777912139893]
    CameraTransform = conf.CAMERA_TRANSFORM
    urdf_path = conf.urdf 
    model_path = conf.modelPath 
    
    


visualizer = visualize.Visualizer(showFloor=True, cameraTF=CameraTransform)

ground_truth_options = {'contact_names': conf.parent_frames, 
                'robot_color': [.3,1.,.3,.2],
                'force_color': [0., 1., 0., .5],
                'force_radius': .002, 
                'force_length': .025,
                'cone_color': [0., 1., 0., .5],
                'cone_length': .02,
                'friction_coeff': conf.mu, 
                'wireframe_mode': "FILL"
                }

sim_options = {'contact_names':conf.parent_frames, 
                'robot_color': [.7,.7, .7,1.],# [.6,.4, 1.,.8],
                'force_color': [0., 0., 1., .8],
                'force_radius': .002, 
                'force_length': .025,
                'cone_color': [1., .0, .0, .8],
                'cone_length': .02,
                'friction_coeff': conf.mu,
                'wireframe_mode': "FILL"
                }


ground_truth_viz = visualize.ConsimVisual("truth", urdf_path, 
                    [model_path], pin.JointModelFreeFlyer(), 
                    visualizer, ground_truth_options, conf)

ground_truth_viz.loadViewerModel()

sim_viz = visualize.ConsimVisual("simulation", urdf_path, 
                    [model_path],pin.JointModelFreeFlyer(), 
                    visualizer, sim_options, conf)

sim_viz.loadViewerModel()
sim_names = [n['name'] for n in SIMU_PARAMS] 

data_truth = data[sim_names[15]]
ground_truth_viz.display(data_truth.q[:,0], data_truth.f[:,:,0])

In [7]:
import os 

# font for drawing inf norm 
fnt = ImageFont.truetype('times.ttf', 15) 
SAVE_INF_NORM = 1 


if(TEST_NAME=='solo-trot'):
    viz_step = 5 
else:
    viz_step = 1
    

for i, simName in enumerate(sim_names):
    data_sim = data[simName]
    if 'imp-eul' in simName:
        data_truth = data['ground-truth-implicit-euler']
    elif 'rk4' in simName:
        data_truth = data['ground-truth-rk4']
    elif 'eul-ex' in simName:
        data_truth = data[ 'ground-truth-euler']
    elif 'expo' in simName:
        data_truth = data['ground-truth-exp']

    
    

    q0 = np.zeros(data_sim.q[:,0].shape)
    q0[2] += .015

    viz_name = simName
    print(viz_name.center(LINE_WIDTH, '#'))
    horizon = data_truth.q.shape[1]
    saving_path = os.getcwd() +"/image_captures/"+TEST_NAME+"/"+viz_name
    ind = 0 
    for t in range(0,horizon,viz_step):
        try:
            sim_viz.display(q0+data_sim.q[:,t],data_sim.f[:,:,t])
            ground_truth_viz.display(q0+data_truth.q[:,t], data_truth.f[:,:,t])
            if SAVE_INF_NORM:
                img = Image.new('RGB', (80,80), color = (255,255,255))
                d = ImageDraw.Draw(img)
                d.text((10,30), "{:.3e}".format(res.err_traj_infnorm[simName][t]), font=fnt, fill=(0, 0, 0))
                img.save(os.getcwd() +"/image_captures/"+TEST_NAME+"/inf_err_"+viz_name+"_{:03d}".format(ind)+".png")
        except:
            sim_viz.display(q0+data_sim.q[:,-1],data_sim.f[:,:,-1])
            ground_truth_viz.display(q0+data_truth.q[:,-1], data_truth.f[:,:,-1])
            if SAVE_INF_NORM:
                img = Image.new('RGB', (80,80), color = (255,255,255))
                d = ImageDraw.Draw(img)
                d.text((10,30), "{:.3e}".format(res.err_traj_infnorm[simName][-1]), font=fnt, fill=(0, 0, 0))
                img.save(os.getcwd() +"/image_captures/"+TEST_NAME+"/inf_err_"+viz_name+"_{:03d}".format(ind)+".png")
            
        
        visualizer.captureFrame(saving_path+"_{:03d}".format(ind)+".png")
        ind += 1 
        time.sleep(1.e-2)

##########################################expo    1 mmm 0###########################################
##########################################expo    2 mmm 0###########################################
##########################################expo    4 mmm 0###########################################
##########################################expo    8 mmm 0###########################################
##########################################expo   16 mmm 0###########################################
##########################################expo   32 mmm 0###########################################
##########################################expo   64 mmm 0###########################################
############################################imp-eul    1############################################
############################################imp-eul    2############################################
############################################imp-eul    4###################################